In [1]:
import ROOT
import numpy
from ROOT import larcv
from matplotlib import pyplot as plt
import pandas

%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Welcome to JupyROOT 6.14/04


In [11]:
# file_name = "/Users/corey.adams/DeepLearnPhysics/SparseEventID/analysis/val_event_id_2d_24nodes_bs4608_nf12_out.root "
file_name = "/Users/corey.adams/DeepLearnPhysics/SparseEventID/analysis/val_event_id_3d_36nodes_bs3024_nf32_out.root"
io_manager = larcv.IOManager()
io_manager.add_in_file(file_name)
io_manager.initialize()

True

    [NORMAL]  <IOManager::prepare_input> Opening a file in READ mode: /Users/corey.adams/DeepLearnPhysics/SparseEventID/analysis/val_event_id_3d_36nodes_bs3024_nf32_out.root
    [NORMAL]  <IOManager::initialize> Prepared input with 1000 entries...


In [12]:
io_manager.get_n_entries()

1000

In [13]:
for p in io_manager.producer_list("meta"):
    print(p)

label_cpi
label_neut
label_npi
label_prot


In [14]:
def convert_to_pandas(io_manager):
    
    # This function reads in the data from larcv, and converts to pandas.  It stores the neutrino energy (true),
    # as well as the true and predicted labels.
    
    # initialize a dataframe:
    df = pandas.DataFrame(
        index = numpy.arange(io_manager.get_n_entries() - 1),
        columns = ["pred_neut", "pred_neut0", "pred_neut1", "pred_neut2",
                   "pred_npi",  "pred_npi0",  "pred_npi1",
                   "pred_cpi",  "pred_cpi0",  "pred_cpi1",
                   "pred_prot", "pred_prot0", "pred_prot1", "pred_prot2",
                   "true_neut", "true_npi", "true_cpi", "true_prot",
                   "energy",]
    )
    
    for i in range(io_manager.get_n_entries()):

        io_manager.read_entry(i)
#         print ("Event ", i)
        
        # get the true labels:
        cpiID = io_manager.get_data("particle", "cpiID")
        npiID = io_manager.get_data("particle", "npiID")
        neutID = io_manager.get_data("particle", "neutID")
        protID = io_manager.get_data("particle", "protID")
        neutrino = io_manager.get_data("particle", "sbndneutrino")
        
        if i != 0:
            df.iloc[i-1]['true_cpi'] = cpiID.as_vector().front().pdg_code()
            df.iloc[i-1]['true_npi'] = npiID.as_vector().front().pdg_code()
            df.iloc[i-1]['true_neut'] = neutID.as_vector().front().pdg_code()
            df.iloc[i-1]['true_prot'] = protID.as_vector().front().pdg_code()
            df.iloc[i-1]['energy'] = neutrino.as_vector().front().energy_init()

        label_cpi  = io_manager.get_data("meta", "label_cpi")
        label_npi  = io_manager.get_data("meta", "label_npi")
        label_prot = io_manager.get_data("meta", "label_prot")
        label_neut = io_manager.get_data("meta", "label_neut")

#         df.iloc[i]['pred_cpi'] = label_cpi.as_vector().front().pdg_code()
        logits_cpi  = label_cpi.get_darray('meta')
        logits_npi  = label_npi.get_darray('meta')
        logits_prot = label_prot.get_darray('meta')
        logits_neut = label_neut.get_darray('meta')

        index = i

        if i != io_manager.get_n_entries() -1 :
            df.iloc[index]["pred_neut"]  = numpy.argmax(logits_neut)
            df.iloc[index]["pred_neut0"] = logits_neut[0]
            df.iloc[index]["pred_neut1"] = logits_neut[1]
            df.iloc[index]["pred_neut2"] = logits_neut[2]

            df.iloc[index]["pred_cpi"]  = numpy.argmax(logits_cpi)
            df.iloc[index]["pred_cpi0"] = logits_cpi[0]
            df.iloc[index]["pred_cpi1"] = logits_cpi[1]

            df.iloc[index]["pred_npi"]  = numpy.argmax(logits_npi)
            df.iloc[index]["pred_npi0"] = logits_npi[0]
            df.iloc[index]["pred_npi1"] = logits_npi[1]

            df.iloc[index]["pred_prot"]  = numpy.argmax(logits_prot)
            df.iloc[index]["pred_prot0"] = logits_prot[0]
            df.iloc[index]["pred_prot1"] = logits_prot[1]
            df.iloc[index]["pred_prot2"] = logits_prot[2]
        
    return df
        
    

In [15]:
df = convert_to_pandas(io_manager)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 998
Data columns (total 19 columns):
pred_neut     999 non-null object
pred_neut0    999 non-null object
pred_neut1    999 non-null object
pred_neut2    999 non-null object
pred_npi      999 non-null object
pred_npi0     999 non-null object
pred_npi1     999 non-null object
pred_cpi      999 non-null object
pred_cpi0     999 non-null object
pred_cpi1     999 non-null object
pred_prot     999 non-null object
pred_prot0    999 non-null object
pred_prot1    999 non-null object
pred_prot2    999 non-null object
true_neut     999 non-null object
true_npi      999 non-null object
true_cpi      999 non-null object
true_prot     999 non-null object
energy        999 non-null object
dtypes: object(19)
memory usage: 156.1+ KB


In [17]:
df

,pred_neut,pred_neut0,pred_neut1,pred_neut2,pred_npi,pred_npi0,pred_npi1,pred_cpi,pred_cpi0,pred_cpi1,pred_prot,pred_prot0,pred_prot1,pred_prot2,true_neut,true_npi,true_cpi,true_prot,energy
0,1,0.328524,0.342951,0.328525,0,0.505525,0.494475,0,0.505139,0.494861,0,0.341877,0.329306,0.328817,1,0,0,0,0.875862
1,2,0.325448,0.325448,0.349104,1,0.486133,0.513867,0,0.509781,0.490219,0,0.348684,0.325686,0.32563,2,1,0,0,0.799547
2,0,0.392066,0.298501,0.309433,0,0.566923,0.433077,1,0.458134,0.541866,0,0.438057,0.282662,0.279281,0,0,1,0,0.771936
3,0,0.354095,0.322757,0.323148,0,0.521151,0.478849,0,0.522105,0.477895,1,0.318805,0.3624,0.318795,0,0,0,1,0.806828
4,0,0.429992,0.272191,0.297817,0,0.537056,0.462944,0,0.600706,0.399294,2,0.25347,0.249033,0.497497,0,0,0,2,1.91603
5,1,0.327095,0.345804,0.327101,0,0.50686,0.49314,0,0.508321,0.491679,1,0.327735,0.344527,0.327738,1,0,0,1,0.643718
6,2,0.319934,0.326914,0.353152,0,0.530215,0.469785,1,0.480219,0.519781,1,0.322661,0.355076,0.322263,2,0,1,1,1.36126
7,0,0.3425,0.327574,0.329926,0,0.512362,0.487638,0,0.511941,0.488059,1,0.325246,0.349462,0.325292,0,0,0,1,0.317034
8,1,0.318072,0.351165,0.330762,0,0.518867,0.481133,0,0.501107,0.498893,0,0.352405,0.324369,0.323226,1,0,1,0,0.956753
9,1,0.326031,0.342672,0.331297,0,0.509271,0.490729,1,0.489812,0.510188,2,0.328629,0.330036,0.341335,2,0,1,2,1.61854


In [18]:
# This quickly calculates the accuracy:
def calculate_accuracy(df):
    for key in ['neut', 'npi', 'cpi', 'prot']:
        accuracy = numpy.mean(df['true_{}'.format(key)] == df['pred_{}'.format(key)])
        print(accuracy)

In [19]:
calculate_accuracy(df)

0.9159159159159159
0.955955955955956
0.9409409409409409
0.913913913913914
